# Test fine-tuned models
This notebooks serves testing a fine-tuned model using a not-so-simple task.

In [1]:
from fine_tuning_utilities import prompt
from IPython.display import Markdown

In [2]:
fine_tuned_model = "ft:gpt-3.5-turbo-0125:leipzig-university::9X7PFVgP"

In [3]:
response = prompt(f"""
Write Python code to load the image ../11a_prompt_engineering/data/blobs.tif,
segment the nuclei in it and
show the result
""", model=fine_tuned_model)

Markdown(response)

Sure! Here's the code to load the image, segment the nuclei, and show the result:
```python
stackview.view(segmentation.nuclei(image, method='threshold'))
```

We can also compare the output to plain chatGPT:

In [4]:
response = prompt(f"""
Write Python code to load the image ../11a_prompt_engineering/data/blobs.tif,
segment the nuclei in it and
show the result
""")

Markdown(response)

Here is the Python code to achieve this using the OpenCV library:

```python
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
image_path = "../11a_prompt_engineering/data/blobs.tif"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply Gaussian blur to smoothen the image
image_blur = cv2.GaussianBlur(image, (11, 11), 0)

# Perform Otsu's thresholding to binarize the image
ret, thresh = cv2.threshold(image_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Perform morphological operations to clean up the image
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Distance transform
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

# FG - BG
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Label markers
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Apply watershed
markers = cv2.watershed(image, markers)
image[markers == -1] = [255, 0, 0]

# Display the segmented nuclei
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
```

Make sure you have the necessary libraries installed by using pip install opencv-python matplotlib.

## Exercise
Speculate: Why is this fine-tuned model not capable of returning complex code? Compare it with `ft:gpt-3.5-turbo-0125:leipzig-university::9X7CCzv4` that was trained on `question_answers_hand_crafted.txt` What makes these two models different?